In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
from sklearn.preprocessing import OneHotEncoder


# Creation of user_df, a DataFrame with informations about Archived Users

users = []

for filepath in glob.iglob('Users/Archived users/*.txt'):
    users.append(filepath[-14:-4])

# print(users)
user_number = len(users)    # number of users

user_df = pd.DataFrame(users, columns = ["Users"])

# Create columns

col_index = []

file = open('Users/Archived users/User_' + str(users[0]) + ".txt")
lines = file.read().split("\n")
ll = len(lines)

for l in range(ll-1):
    sl = lines[l].split(": ", 1)
    col_index.append(sl[0])

# print(col_index)

# Create values

values_list = []
values = []

for i in range(user_number):
    file = open('Users/Archived users/User_' + str(users[i]) + ".txt")
    lines = file.read().split("\n")
    for l in range(ll-1):
        sl = lines[l].split(": ", 1)
        v = sl[1]
        values.append(v)
    values_list.append(values)
    values = []

# Insert values in user_df

value_df = pd.DataFrame(values_list, columns = col_index)

# print (value_df)

user_df = pd.concat([user_df, value_df], axis = 1)

# print(user_df)

# Creation of tappydata_df, a DataFrame with informations about Archived Data

tappydata = []

for filepath in glob.iglob('Data/Tappy Data/*.txt'):
    file = open(filepath)
    lines = file.read().split("\n")
    for l in range(len(lines)-1):
        x = lines[l]
        x = x.split("\t")
        tappydata.append(x)

tappydata_df = pd.DataFrame(tappydata, columns = ["Users", "Date", "Datetime", "TapPosition", "Hold", "Movement", "Latency", "Flight", "*"])

# print(tappydata_df)

all_userdata_df = pd.merge(tappydata_df, user_df, how='inner', left_on='Users', right_on='Users')


In [18]:
#

all_userdata_df = all_userdata_df[all_userdata_df.Impact != "Severe"]
all_userdata_df = all_userdata_df[all_userdata_df.Impact != "Medium"]

all_userdata_df = all_userdata_df[all_userdata_df.Levadopa == "False"]

# counts = all_userdata_df['Users'].value_counts()
# print(counts)

In [23]:
counts_parkinson = all_userdata_df[all_userdata_df["Parkinsons"] == "True"]
counts_parkinson.nunique()

# quindi ciamo 34 malati e 55 sani

counts = all_userdata_df['Users'].value_counts()
avg = all_userdata_df.shape[0] / len(counts)
#print(avg)  # numero medio di tap per ogni utente

#counts.plot(kind = "hist")
#counts.plot(kind = "hist", range = [0, 50])

# togliere da dopo 1000 a ???
# se togliamo i 3 che fanno tanti tap (3/4/5/600 k) Che succede??? quei dati influenzano troppo?

all_userdata_df = all_userdata_df[all_userdata_df['Users'].isin(counts[counts > 1000].index)]
all_userdata_df = all_userdata_df[all_userdata_df['Users'].isin(counts[counts < 200000].index)]

#all_userdata_df.head()

# non leviamo i tap con lo spazio !!!

#all_userdata_df = all_userdata_df.drop(["Users", "Date", "Datetime", "Flight", "*", "Tremors", "DiagnosisYear", "Sided", "UPDRS","Impact","Levadopa", "DA","MAOB", "Other"], axis=1)

# completata la pulizia

             Users        Date Datetime   TapPosition Hold Movement Latency  \
409197  6LB9FQABZQ  6LB9FQABZQ   161029  16:17:54.109    R   0121.1      AD   

        Flight * BirthYear  ... Parkinsons Tremors DiagnosisYear Sided  \
409197  0375.0        1956  ...      False   False        ------  None   

             UPDRS   Impact Levadopa     DA   MAOB  Other  
409197  Don't know   ------    False  False  False  False  

[1 rows x 21 columns]


In [16]:
#all_userdata_df.head()

# le colonne ora sono: tap position (L/R), Hold, Movement, BirthYear, Gender, Parkinsons (T/F)
# rimuovo gli utenti senza data di nascita, perche mi serve per calcolare l'eta

all_userdata_df = all_userdata_df[(all_userdata_df.BirthYear != "")]

# normalizzazione
# L/R - Gender - Parkinson : devono diventare campi binari del tipo 0/1
# movement : lo codifichiamo con onehot dividendolo in 4 colonne
# latency - hold : sono dei float
# age (ottenuta da 2017 - birthyear) : int

all_userdata_df = all_userdata_df.rename(columns = {"TapPosition" : "RightTap"})

#all_userdata_df.drop(all_userdata_df.loc[all_userdata_df['Hold']=='R'].index, inplace=True)
#all_userdata_df.drop(all_userdata_df.loc[all_userdata_df['Hold']=='16:15:50.375'].index, inplace=True)

# RICONTROLLARE PERCHE ESCE LA R SULLA COLONNA HOLD (RIGA 409197)

from sklearn.preprocessing import OneHotEncoder
all_userdata_df.RightTap.replace(['R', 'L', 'S'], [0, 1, 2], inplace = True)
all_userdata_df.head()
# va fatto il onehot
all_userdata_df.Hold = all_userdata_df.Hold.astype(float)
# TOGLIERE COMMENTO E CAPIRE PERCHE NON FUNZIONA (CANT CONVERT STRING TO FLOAT)

#counts_ss = all_userdata_df[all_userdata_df["Movement"] == "SS"]

all_userdata_df.Movement.replace(['LL', 'LR', 'RR', 'RL', 'LS', 'SL', 'RS', 'SR', 'SS'], [0, 1, 2, 3, 4, 5, 6, 7, 8], inplace = True)
# va fatto il onehot

all_userdata_df.Latency = all_userdata_df.Latency.astype(float)
# TOGLIERE COMMENTO E CAPIRE PERCHE NON FUNZIONA (CANT CONVERT STRING TO FLOAT)

all_userdata_df.BirthYear = all_userdata_df.BirthYear.astype(int)
all_userdata_df.BirthYear = 2017 - all_userdata_df.BirthYear

all_userdata_df.Gender.replace(['Male', 'Female'], [1, 0], inplace=True)

all_userdata_df.Parkinsons.replace(['True', 'False'], [1, 0], inplace=True)


counts_Rhold = all_userdata_df[all_userdata_df["Hold"] == '07:54:33.016']
print(counts_Rhold)


#counts_Rhold = all_userdata_df[all_userdata_df["Hold"] == "R"]
#print(counts_Rhold)

# parte dedicata al onehot di RightTap e Movement
OneHotRightTap = pd.get_dummies(all_userdata_df.RightTap)
OneHotMovement = pd.get_dummies(all_userdata_df.Movement)

OneHotMovement = OneHotMovement.rename(columns = {0: "LL", 1: "LR", 2: "RR", 3: "RL", 4: "LS", 5: "SL", 6: "RS", 7: "SR", 8: "SS"})
all_userdata_df = all_userdata_df.drop(["Movement"], axis=1)

OneHotRightTap = OneHotRightTap.rename(columns = {0: "R", 1: "L", 2: "S"})
all_userdata_df = all_userdata_df.drop(["RightTap"], axis=1)

all_userdata_df = pd.concat([all_userdata_df, OneHotRightTap, OneHotMovement], axis=1)

#all_userdata_df = all_userdata_df.reindex_axis(["RightTap","LL","LR","RR","RL",'LS', 'SL', 'RS', 'SR', 'SS', "Latency","Hold","BirthYear","Gender","Parkinsons"], axis=1)


        RightTap          Hold Movement Latency  BirthYear  Gender  Parkinsons
2293514   160912  07:54:33.016        R      LR         71       0           1
